In [1]:
%pip install pandas
%pip install pygame

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Myranda\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Myranda\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import pygame
import sys
import pandas as pd

# Load the GAIA dataset
file_path = 'map_object_list.csv'
gaia_data = pd.read_csv(file_path)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption('Virtual Telescope')
font = pygame.font.SysFont(None, 24)

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
HIGHLIGHT = (255, 0, 255)

# Starting position and zoom factor of the telescope
telescope_glon = 0
telescope_glat = 0
zoom_factor = 1.0
base_star_size = 2

# Selected star information
selected_star_info = None
highlighted_star_info = None

# Function to convert galactic coordinates to screen coordinates
def convert_to_screen_coords(glon, glat, telescope_glon, telescope_glat, zoom_factor, screen_width, screen_height):
    x = (glon - telescope_glon + 360) % 360 - 180  # Handle wrap-around at 0/360 degrees
    y = glat - telescope_glat
    x = (x + 180) * screen_width / 360.0 * zoom_factor
    y = (90 - y) * screen_height / 180.0 * zoom_factor
    return int(x), int(y)

# Function to check if a point is near a star
def is_near(x1, y1, x2, y2, threshold):
    return abs(x1 - x2) < threshold and abs(y1 - y2) < threshold

# Function to get color based on distance
def get_star_color(distance):
    if distance < 1000:
        return RED
    elif distance < 2000:
        return YELLOW
    elif distance < 3000:
        return GREEN
    else:
        return WHITE

# Function to get star size based on distance
def get_star_size(distance):
    if distance < 1000:
        return 5
    elif distance < 2000:
        return 4
    elif distance < 3000:
        return 3
    else:
        return 2

# Main loop
while True:
    highlighted_star_info = None
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = event.pos
            for index, star in gaia_data.iterrows():
                glon, glat, name, distance = star['glon'], star['glat'], star['name'], star['distance (parsecs)']
                x, y = convert_to_screen_coords(glon, glat, telescope_glon, telescope_glat, zoom_factor, 800, 600)
                if is_near(mouse_x, mouse_y, x, y, max(5, int(base_star_size * zoom_factor))):
                    selected_star_info = f"Name: {name}\nGlon: {glon}\nGlat: {glat}\nDistance: {distance} parsecs"
                    break

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        telescope_glon -= 1 / zoom_factor
    if keys[pygame.K_RIGHT]:
        telescope_glon += 1 / zoom_factor
    if keys[pygame.K_UP]:
        telescope_glat += 1 / zoom_factor
    if keys[pygame.K_DOWN]:
        telescope_glat -= 1 / zoom_factor
    if keys[pygame.K_EQUALS] or keys[pygame.K_PLUS]:  # Zoom in using Shift +
        zoom_factor *= 1.1
    if keys[pygame.K_MINUS]:  # Zoom out using Shift -
        zoom_factor /= 1.1

    # Keep telescope_glat within valid range
    if telescope_glat > 90:
        telescope_glat = 90
    if telescope_glat < -90:
        telescope_glat = -90

    # Clear the screen
    screen.fill(BLACK)

    # Draw stars
    for index, star in gaia_data.iterrows():
        glon, glat, name, distance = star['glon'], star['glat'], star['name'], star['distance (parsecs)']
        x, y = convert_to_screen_coords(glon, glat, telescope_glon, telescope_glat, zoom_factor, 800, 600)
        if 0 <= x < 800 and 0 <= y < 600:  # Draw only if within screen bounds
            star_color = get_star_color(distance)
            star_size = get_star_size(distance)  # Adjust star size based on distance
            pygame.draw.circle(screen, star_color, (x, y), star_size)

            # Highlight star on hover
            mouse_x, mouse_y = pygame.mouse.get_pos()
            if is_near(mouse_x, mouse_y, x, y, max(5, int(base_star_size * zoom_factor))):
                pygame.draw.circle(screen, HIGHLIGHT, (x, y), star_size + 2)
                highlighted_star_info = f"Name: {name}\nGlon: {glon}\nGlat: {glat}\nDistance: {distance} parsecs"

    # Display selected star information
    if selected_star_info:
        info_surface = font.render(selected_star_info, True, YELLOW)
        screen.blit(info_surface, (10, 10))
    
    # Display highlighted star information
    if highlighted_star_info:
        highlight_surface = font.render(highlighted_star_info, True, HIGHLIGHT)
        screen.blit(highlight_surface, (10, 40))

    pygame.display.flip()

SystemExit: 

C:\Users\Myranda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
